## Selenium (My balls)

In [55]:
import requests
from bs4 import BeautifulSoup as bs
import pandas as pd
import time
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

# Define the URL and user-agent to mimic a real browser
url = 'https://www.sciencedirect.com/science/article/pii/S0927537123000404'
user_agent = (
    "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/100.0.1234.0 Safari/537.36"
)

# Use a non-headless browser with a custom user-agent
options = webdriver.ChromeOptions()
options.add_argument(f"user-agent={user_agent}")
options.add_argument('--headless=new')

## Find the first article url in every issue

In [121]:
def index_volumes(journal,vol):
    url_base = "https://www.sciencedirect.com/journal/" + str(journal) + "/vol/"
    url_list = []
    for i in range(40): #EDIT
        url_pre = url_base + str(vol-i) + "/suppl/C"
        url_list.append(url_pre)
    
    driver = webdriver.Chrome(options=options)


    first_page_url = []
    for it, url in enumerate(url_list):
        print(f"Processing URL {it+1}/{len(url_list)}: {url}")
        try:
            driver.get(url)
        
            # Wait for the first link with class "article-content-title" to be clickable
            wait = WebDriverWait(driver, 10)
            link = wait.until(EC.element_to_be_clickable((By.CSS_SELECTOR, 'a.article-content-title')))
        
            # Click the link
            link.click()
        
            # Get the current URL of the page
            first_page_url.append(driver.current_url)
        except Exception as e:
            print(f"An error occurred for URL {url}")
            
    return first_page_url
    
    


In [122]:
def scrape_article(first_page_url):
    journal = []
    for it, url in enumerate(first_page_url):
        print(f"Processing URL {it+1}/{len(first_page_url)}: {url}")

        driver = webdriver.Chrome(options=options)
        driver.get(url)

        i = 0

        page_source = []

        while i < 50:
            time.sleep(5)  # Wait for a longer time between requests


            print(i)
            page_source.append(driver.page_source)

            try:
                # Wait for the next button to become clickable before clicking it
                wait = WebDriverWait(driver, 10)
                next_button = wait.until(EC.element_to_be_clickable((By.CSS_SELECTOR, 'a.button-alternative-icon-right')))
                next_button.click()

                i += 1
            except Exception as e:
                print("Error with article")
                break
    
        journal.append(page_source)
    # Close the browser
    driver.quit()
    return journal

In [158]:
scrape_aa = [('economic-modelling', 127),('economics-letters',231),
           ('journal-of-econometrics',236),('journal-of-development-economics',165),
           ('journal-of-applied-economics',20),('the-journal-of-socio-economics',48),('journal-of-economic-theory',212)]

scrape1 = [('economic-modelling', 127)]
scrape2 = [('economics-letters',219),('journal-of-econometrics',236)]
url_full = []
for item in scrape1:
    journal_name, last_vol = item
    print(journal_name)
    url_list_pre = index_volumes(journal_name, last_vol)
    url_full.extend(url_list_pre)


print(url_full)

articles = scrape_article(url_full)

# Write the data from the 'articles' list into the CSV file
with open(csv_file, 'w', newline='') as file:
    csv_writer = csv.writer(file)
    csv_writer.writerows(articles)

print(f'CSV file "{csv_file}" created successfully.')

economic-modelling
Processing URL 1/40: https://www.sciencedirect.com/journal/economic-modelling/vol/127/suppl/C
Processing URL 2/40: https://www.sciencedirect.com/journal/economic-modelling/vol/126/suppl/C
Processing URL 3/40: https://www.sciencedirect.com/journal/economic-modelling/vol/125/suppl/C
Processing URL 4/40: https://www.sciencedirect.com/journal/economic-modelling/vol/124/suppl/C
Processing URL 5/40: https://www.sciencedirect.com/journal/economic-modelling/vol/123/suppl/C
Processing URL 6/40: https://www.sciencedirect.com/journal/economic-modelling/vol/122/suppl/C
Processing URL 7/40: https://www.sciencedirect.com/journal/economic-modelling/vol/121/suppl/C
Processing URL 8/40: https://www.sciencedirect.com/journal/economic-modelling/vol/120/suppl/C
Processing URL 9/40: https://www.sciencedirect.com/journal/economic-modelling/vol/119/suppl/C
Processing URL 10/40: https://www.sciencedirect.com/journal/economic-modelling/vol/118/suppl/C
Processing URL 11/40: https://www.scienc

KeyboardInterrupt: 

In [130]:
journal_temp = journal


IOPub data rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_data_rate_limit`.

Current values:
ServerApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
ServerApp.rate_limit_window=3.0 (secs)



## Strukturer Data

In [152]:
def create_df(page_source):
    df_data = []

    for j in range(len(page_source)):
        authors_dict = {}  # Create a new dictionary for each row
        journal_name = []
        soup = bs(page_source[j], 'html.parser')
        try:
            # Find the element with ID "sec0001"
            element_with_id = soup.find('section', id='sec0001')
            
            # Extract and store the text content in the "introduction" variable
            introduction = element_with_id.get_text(strip=True)
        except AttributeError:
            introduction = None  # Handle case when element is not found
        
        # Find all <span> elements with class="given-name"
        author_spans = soup.find_all('span', class_='given-name')

        # Loop through the author spans and populate the dictionary
        for i, author_span in enumerate(author_spans):
            try:
                if i < 5:
                    authors_dict[f'Author {i+1}'] = author_span.get_text().split()[0]
                    if "." in authors_dict[f'Author {i+1}']:
                        author_name = "Unknown"
                        authors_dict[f'Author {i+1}'] = author_name
                else:
                    break
            except AttributeError:
                authors_dict[f'Author {i+1}'] = "None"
        
        try:
            j_name = soup.find('a', class_="publication-title-link").get_text()
            journal_name.append(str(j_name))
        except:
            journal_name.append("Error no journal name")
                               
        
        
        # Append the data for this row to the list of rows
        df_data.append({'Introduction': introduction, **authors_dict, 'Journal': journal_name})

    df = pd.DataFrame(df_data)

    return df

In [153]:
def combine_dataframes(list_of_lists):
    combined_df = pd.DataFrame()  # Initialize an empty DataFrame to store the combined data

    for sublist in list_of_lists:
        # Apply create_df function to the sublist and get a dataframe
        df = create_df(sublist)
        
        # Concatenate the current dataframe with the combined dataframe
        combined_df = pd.concat([combined_df, df], ignore_index=True)

    return combined_df

#full_df = combine_dataframes(journal)
full_df = combine_dataframes(journal[1:2])


In [154]:
full_df[columns_to_replace] = full_df[columns_to_replace].fillna("None")
df = full_df
df

,Introduction,Journal,Author 1,Author 2,Author 3
0,None,[Labour Economics],None,None,None
1,1. IntroductionPublic subsidies to private bus...,[Labour Economics],Federico,Filippo,Paolo
2,1. IntroductionAccording to theEuropean Agency...,[Labour Economics],Martin,None,None
3,1. IntroductionFinancial support to families w...,[Labour Economics],Erica,Olof,Håkan
4,1. IntroductionThroughout the twentieth centur...,[Labour Economics],Ria,Anne,None
5,1. IntroductionDespite a considerable decline ...,[Labour Economics],Omar,Lennart,None
6,1. IntroductionMany developed countries have a...,[Labour Economics],Teresa,Clara,Annekatrin
7,1. IntroductionThe Covid-19 pandemic triggered...,[Labour Economics],Gaetano,Domenico,Salvatore
8,1. IntroductionAbsenteeismmay severely depress...,[Labour Economics],Sam,Thomas,Jan
9,1. IntroductionConsistent across a large body ...,[Labour Economics],Emelie,Jenny,Richard


In [134]:
df.to_csv('output.csv', index=False)

In [155]:
import gender_guesser.detector as gender

detector = gender.Detector()

def predict_gender_first_name_api(name):
    api_url = 'https://api.genderize.io?name=' + str(name)
    print(api_url)
    response = requests.get(api_url)
    # Check if the request was successful (200 OK)
    if response.status_code == 200:
        data = response.json()  # Parse the response content as JSON
        try:
            gender = data["gender"].capitalize()
            probability = data["probability"]
            if data["gender"] in {'male', 'female'}:
                return data["gender"]
            else:
                return "Unknown"
            
        except Exception as e:
            print(f"Error predicting gender for {name}: {e}")
            return "Unknown"
    else:
        print(f"Request failed. Status code: {response.status_code}")
        return "Unknown"


# Function to get the first name from a full name
def get_first_name(full_name):
    if full_name is not None:
        return full_name.split()[0]
    else:
        return "None"

# Function to predict gender for the first name
def predict_gender_first_name(name):
    gender = detector.get_gender(name)
    if name == "None":
        return "None"
    elif "." in name:
        return "None"
        print(name)
    elif gender == "unknown":
        gender_api = predict_gender_first_name_api(name)
        return gender_api
    else:
        return gender

# Apply gender prediction to the first name in each author column and create new gender columns
df['Gender_Author 1'] = df['Author 1'].apply(predict_gender_first_name)
df['Gender_Author 2'] = df['Author 2'].apply(predict_gender_first_name)
df['Gender_Author 3'] = df['Author 3'].apply(predict_gender_first_name)
df['Gender_Author 4'] = df['Author 4'].apply(predict_gender_first_name)
df['Gender_Author 5'] = df['Author 5'].apply(predict_gender_first_name)

#df.drop(columns=['Gender_Author 1', 'Gender_Author 2', 'Gender_Author 3'], inplace=True)

# Map the gender predictions to more readable form
df['Gender_Author 1'] = df['Gender_Author 1'].map({
    'male': 'Male',
    'female': 'Female',
    'unknown': 'Unknown',
})
df['Gender_Author 2'] = df['Gender_Author 2'].map({
    'male': 'Male',
    'female': 'Female',
    'unknown': 'Unknown',
})
df['Gender_Author 3'] = df['Gender_Author 3'].map({
    'male': 'Male',
    'female': 'Female',
    'unknown': 'Unknown',
})
df['Gender_Author 4'] = df['Gender_Author 4'].map({
    'male': 'Male',
    'female': 'Female',
    'unknown': 'Unknown',
})
df['Gender_Author 5'] = df['Gender_Author 5'].map({
    'male': 'Male',
    'female': 'Female',
    'unknown': 'Unknown',
})


https://api.genderize.io?name=Raül


In [156]:
# Function to get the article gender based on author genders
def get_article_gender(row):
    genders = [row[f'Gender_Author {i+1}'] for i in range(3) if not pd.isnull(row[f'Gender_Author {i+1}'])]
    if len(genders) == 0:
        return 'Unknown'
    elif all(gender == 'Male' for gender in genders):
        return 'Male'
    elif all(gender == 'Female' for gender in genders):
        return 'Female'
    else:
        return 'Mix'

# Apply the function to create the "Article_Gender" column
df['Article_Gender'] = df.apply(get_article_gender, axis=1)

# Display the updated DataFrame
print(df)

                                         Introduction             Journal  \
0                                                None  [Labour Economics]   
1   1. IntroductionPublic subsidies to private bus...  [Labour Economics]   
2   1. IntroductionAccording to theEuropean Agency...  [Labour Economics]   
3   1. IntroductionFinancial support to families w...  [Labour Economics]   
4   1. IntroductionThroughout the twentieth centur...  [Labour Economics]   
5   1. IntroductionDespite a considerable decline ...  [Labour Economics]   
6   1. IntroductionMany developed countries have a...  [Labour Economics]   
7   1. IntroductionThe Covid-19 pandemic triggered...  [Labour Economics]   
8   1. IntroductionAbsenteeismmay severely depress...  [Labour Economics]   
9   1. IntroductionConsistent across a large body ...  [Labour Economics]   
10  1. IntroductionThe compatibility of labour mar...  [Labour Economics]   
11  1. IntroductionAfter dropping steeply at the s...  [Labour Economics]   